*Code by Claudia Corradino-INGV Catania.*\
*Reference: Corradino, C., Jouve, P., La Spina, A., & Del Negro, C. (2024). Monitoring Earth's atmosphere with Sentinel-5 TROPOMI and Artificial Intelligence: Quantifying volcanic SO2 emissions. Remote Sensing of Environment, 315, 114463.*

# *GEE authentication*
Accessing Google Earth Engine

In [1]:
import ee
# Trigger the authentication flow.
ee.Authenticate()
# Initialize the library.
ee.Initialize(project='victor-corradino')



# *Libraries*

In [2]:
!pip install geemap
import geemap
from datetime import datetime
import openpyxl
import pandas as pd
import matplotlib.pyplot as plt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.7 MB/s eta 0:00:00


#*Functions Definition*

In [3]:
def CountBands(image):
  return image.set('count',image.bandNames().length())

In [4]:
def InputPreparation(image):
  #selecting needed GLCM bands form inital 18 ones
  bands_glcm = ['glcm_contrast','glcm_corr','glcm_var','glcm_savg']
  #bands used for Classification + select(add) bands from glcm
  bands = ee.List(['SO2_column_number_density','so2_15km','sd','entropy']).cat(ee.List(bands_glcm))

  # Kernel for Stdv and entropy
  bigKernel = ee.Kernel.square(
      radius = 4,
      units = 'pixels')

  #Kernel for Convolution
  l3 = [-0.4444,-0.4444, -0.1111, -0.4444,-0.4444]

  #Center of Kernel is zero
  c3 = [-0.1111, -0.1111,3.222, -0.1111,-0.1111]

  #Assemble a list of lists, the 9x9 kernel weights as a 2D matrix
  lists3 = [l3,l3,c3,l3,l3]
  kernel3 = ee.Kernel.fixed(5,5,lists3)

  #Kernel for focalMin/focalMax
  kernel = ee.Kernel.circle(radius = 2)

  ###First training image : '2022-07-27', '2022-07-28'
    #median to take average pixel value from remaning images of collection, multiply to convert into dobson unit
  SO2_at_1km = image.select('SO2_column_number_density').multiply(10000/4.4615)\

  SO2_at_15km = image.select('SO2_column_number_density_15km').multiply(10000/4.4615)\

  #Compute standard derivation (Texture)
  sd = SO2_at_1km.reduceNeighborhood(
   reducer= ee.Reducer.stdDev(),
   kernel= bigKernel
   ).reproject('EPSG:4326',None,3500)

  #Compute Entropy (Texture)
  intso2=SO2_at_1km.multiply(10000).int()
  entropy=intso2.entropy(bigKernel).reproject('EPSG:4326',None,3500)

  #Compute Graylevel Co-occurence matrix
  glcm = intso2.rename('glcm').glcmTexture(size= 1).reproject('EPSG:4326',None,3500).select(bands_glcm)#vor select!
  sd1 = sd.unmask(-100);
  entropy1 = entropy.unmask(-100);
  glcm1 = glcm.unmask(-100);

  input = SO2_at_1km.unmask(-100).rename('SO2_column_number_density').addBands(SO2_at_15km.rename('so2_15km')).addBands(sd1.rename('sd')).addBands(entropy1.rename('entropy')).addBands(glcm1)

  input = input.set('Date', image.get('Date')).set('system:time_start', image.get('system:time_start'))

  date=image.get('system:time_start')
  start = ee.Date(date)
  end = start.advance(1, 'day');
  dataset =ee.ImageCollection('COPERNICUS/S5P/NRTI/L3_CLOUD').filterDate(start,end).filterBounds(study_zone).select('cloud_top_height').max().unmask(-100)
  return input.addBands(dataset.rename('altitude'))


In [5]:
def apply_classifier(image3,ee_classifier):
  alt=image3.select('altitude')
  image2=image3.select(['SO2_column_number_density','so2_15km','sd','entropy','glcm_contrast','glcm_corr','glcm_var','glcm_savg']);
  image=image2.subtract(means).divide(std)
  so2_plume_15km=image3.select('so2_15km');
  so2_plume=image3.select('SO2_column_number_density')
  so2_median_density = so2_plume.unmask(-100);
  so2_classify_mask = image.classify(ee_classifier)
  so2_classify = so2_median_density.updateMask(so2_classify_mask)

  # Post-treatment : Apply a closing (dilatation = focalMax then erosion = focalMin)
  # Define a circle kernel of 1 pixel radius (adjust if needed, try 1.5)
  kernel = ee.Kernel.circle(
      radius = 1,
      units = 'pixels')
  connectivity=50;
  #Closing is better than opening to fullfil the plume and then apply PixelCount
  #2 iterations of dilatation and 3 of erosion seems to give the best results (adjust if needed)
  closed = so2_classify_mask.focalMax(
   kernel= kernel,
   iterations= 2).focalMin(
   kernel= kernel,
   iterations= 3).reproject('EPSG:4326',None,3500);
  so2_closed = so2_median_density.updateMask(closed)

  # Connectivity condition : to delete isolated pixels and keep only pixels part of the plume
  # ConnectedPixelCount returns a connection score of the SO2-positive pixels
  connexions = closed.selfMask().connectedPixelCount(ee.Number(connectivity).add(1)).reproject('EPSG:4326',None,3500);
  proxy = -999
  connexions = connexions.unmask(proxy)

  # Only keep the pixels that have a connectivity high enough
  # Score connectivity of 80 seems to be a good value, adjust if needed
  groupesPixels = connexions.gt(connectivity);
  positive_pixel = groupesPixels.reproject('EPSG:4326',None,3500);

  # Update the mask to the so2 map
  so2_plume = so2_median_density.updateMask(positive_pixel)
  # Update the mask to the so2 map
  so2_plume_15km = image.select('so2_15km').updateMask(positive_pixel)

  m2=positive_pixel.multiply(alt)
  m3=m2.updateMask(m2.gt(-100));
  mmean=ee.Image(ee.Number(m2.reduceRegion(reducer= ee.Reducer.max(),geometry= study_zone,scale= res,maxPixels= 1e13).values().get(0)));
  altcorreted=(positive_pixel.multiply(alt)).selfMask();
  m=m2.where(m2.eq(-100),mmean)
  so2_corrected=so2_plume_15km.where(so2_plume.gte(0),(so2_plume_15km.subtract(so2_plume)).multiply(m.subtract(15000)).divide(14000).add(so2_plume_15km))#
  st2=ee.Geometry.Polygon([[[158.6128787089055, 55.34598773699004],[158.6128787089055, 53.8505541387213],[164.4246462870305, 53.8505541387213],[164.4246462870305, 55.34598773699004]]])#, null, false);

  # Convert VCD from DU to mol/m2 (1DU=4.4615*10^-4mol/m2)
  kton=so2_corrected.updateMask(so2_plume.gte(0)).multiply(4.4615/10000).multiply(ee.Image.pixelArea()).multiply(0.000001*64*0.001).reduceRegion(
          reducer= ee.Reducer.sum(),
          geometry= st2,#so2_corrected.geometry(),
          scale= res,
          maxPixels= 1e13
  ).values().get(0);

  output = so2_plume.addBands(so2_plume_15km).addBands(positive_pixel.rename('classification')).addBands(image3.select('altitude')).addBands(so2_corrected.rename('SO2corrected')).set('kton',kton).set('Date', image3.get('Date')).set('system:time_start', image3.get('system:time_start'))

      #m.rename('masked_alt')).addBands(so2_corrected.rename('SO2corrected')).addBands(alt.rename('altitude'))
      #.set('kton',kton).set('Total_mass_corrected',total_mass111).set('Total_mass_1km',total_mass1).set('Total_mass_15km',total_mass15)//.addBands(ee.Image.constant(total_mass).rename('Total_mass'))

  return output
  #return classified.set('system:time_start', image.get('system:time_start'))

In [6]:
# Function to generate a list of the dates of the time series
def dayOffset(day):
  return startDate.advance(day, 'day')

def generateDateList(startDate, endDate):
  #Calculate the date range in milliseconds
  timeDiff = endDate.millis().subtract(startDate.millis());
  #Convert the difference in days
  daysDiff = timeDiff.divide(1000 * 60 * 60 * 24);
  dateList = ee.List.sequence(0, daysDiff.toInt());
  #Calculate the date from the startDate
  dateRange = dateList.map(dayOffset);
  return dateRange;

In [7]:
def One_image_per_dayfun(date):
    start = ee.Date(date)
    end = start.advance(1, 'day')

    filtered = raw.filterDate(start, end).filterBounds(study_zone)
    first_image = filtered.first()

    # Se non c'è immagine, ritorna un'immagine vuota (nulla)
    return ee.Algorithms.If(
        first_image,
        filtered \
            .select(['SO2_column_number_density', 'SO2_column_number_density_15km']) \
            .median() \
            .clip(study_zone) \
            .set('system:time_start', first_image.get('system:time_start')) \
            .set('Date', ee.Date(first_image.get('system:time_start')).format('dd/MM/YYYY')),
        None
    )

 # *Input*

*Insert Volcano Coordinates and Dates of Interest*

In [8]:
longitude =   160.28357808747296 #@param {type: "raw"},
latitude =  54.61140794839813 #@param {type: "raw"}
radius= 500000 #@param {type: "raw"}
volcano = ee.Geometry.Point([longitude,latitude])
study_zone = volcano.buffer(radius)
res=1113.2

start_day = '2025-07-10' #@param {type: "raw"}
end_day = '2025-09-30' #@param {type: "raw"}
startDate = ee.Date(start_day);
endDate = ee.Date(end_day)


*Model Input preparation*

In [9]:
raw=ee.ImageCollection("COPERNICUS/S5P/NRTI/L3_SO2").select('SO2_column_number_density','SO2_column_number_density_15km').filterBounds(study_zone).filterDate(startDate,endDate.advance(1, 'day'))
dateList = generateDateList(startDate, endDate);
One_image_per_day = ee.ImageCollection(dateList.map(One_image_per_dayfun)).map(CountBands)

s5 = One_image_per_day \
    .filter(ee.Filter.eq('count', 2)) \
    .filter(ee.Filter.neq('Date', '23/03/2025')) \
    .map(InputPreparation)

timestamps = One_image_per_day.aggregate_array('system:time_start').getInfo()
date_strings = [datetime.utcfromtimestamp(ts / 1000).strftime('%Y-%m-%d') for ts in timestamps]
print(s5.size().getInfo())

27


#*ML Model*

*Load the trained classifier and normalization coefficients*

In [10]:
assetId = 'users/claudiacorradino/RF_tropomi'
means = ee.Image('users/claudiacorradino/means_tropomi');
std = ee.Image('users/claudiacorradino/std_tropomi');

*Apply the model*

In [11]:
ee_classifier = ee.Classifier.load(assetId)
classified = s5.map(lambda image: apply_classifier(image, ee_classifier))
ktonList = classified.aggregate_array('kton');

#*Results*

*Interactive Map Visualization*

In [13]:
# Calcola bounding box dinamico dall'immagine
geom = classified.first().geometry()
bounds = geom.bounds().getInfo()['coordinates'][0]

vis_params = {
  'min': -0.0008*10000/4.4615,
  'max': 0.005*10000/4.4615,
  'palette': ['blue', 'purple', 'cyan', 'green', 'yellow', 'red'],
  'opacity': 1,
};

geemap.update_package()
Map = geemap.Map(center=(latitude,longitude), zoom=6)
Map.add_basemap("HYBRID")
Map.add_layer(classified.select('SO2corrected').first(), vis_params, ' RF')
Map.add_layer(ee.Geometry.Polygon([bounds]))

Map


Unzipping geemap-master.zip ...
Data downloaded to: /root/Downloads/geemap-master

Please comment out 'geemap.update_package()' and restart the kernel to take effect:
Jupyter menu -> Kernel -> Restart & Clear Output


Map(center=[54.61140794839813, 160.28357808747296], controls=(WidgetControl(options=['position', 'transparent_…

*Visualize and export*


*Bar plot*

In [ ]:
#Lista di immagini
image_list = classified.toList(classified.size())

#Creazione file Excel
workbook = openpyxl.Workbook()
worksheet= workbook.active
worksheet.append(['Data', 'kton'])

#Estrazione valori in kton
for i in range(image_list.size().getInfo()):
  image = ee.Image(image_list.get(i))
  kton_value = image.get('kton').getInfo()
  date_value = image.get('Date').getInfo()
  worksheet.append([date_value, kton_value])
  print(f"Data: {date_value}, kton: {kton_value}")


#Salvataggio file
file_name = 'kton_dati.xlsx'
workbook.save(file_name)
print("Dati salvati in:", file_name)

#Scaricare file excel
from google.colab import files
files.download('kton_dati.xlsx')

#Leggere il file Excel
df = pd.read_excel('kton_dati.xlsx')
df['Data'] = pd.to_datetime(df['Data'], format='%d/%m/%Y')
df = df.sort_values('Data')

#Creare bar plot
plt.figure(figsize=(10, 6))
plt.bar(df['Data'], df['kton'], color='skyblue')
plt.xlabel('Data')
plt.ylabel('kton')
plt.title('Valori di SO2 in kton per immagine')
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', linestyle='--', alpha=0.3)
plt.tight_layout()

#Salvare bar plot
plt.savefig('bar_plot.png')

#Visualizzare bar plot
plt.show()

#Scaricare bar plot
from google.colab import files
files.download('bar_plot.png')


*Map*

In [ ]:
# Stampa coordinate del bounding box
print("🗺️ Coordinate del bounding box:")
for i, coord in enumerate(bounds):
    print(f"  Vertice {i+1}: {coord}")

# Crea geometria Earth Engine per l’export
bbox = ee.Geometry.Polygon([bounds])

geemap.ee_export_image(
    classified.first().select('SO2corrected'), filename='SO2_corrected.tif', scale=5000, region=bbox, file_per_band=False
)